In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('../raw_data/BPE_T1.csv')

####  Note sur Lambert
L'expression des coordonnées dans ce système est tridimensionnelle sous forme de longitudes, latitudes et hauteurs ellipsoïdales. 

In [3]:
data.head()

,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE
0,93,6,6088,06088_0801,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport
1,93,6,6088,06088_1005,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé
2,93,6,6088,06088_1005,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme
3,93,6,6088,06088_3801,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a
4,93,6,6088,06088_3801,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a


In [4]:
data.shape

(11857, 11)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11857 entries, 0 to 11856
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   REG         11857 non-null  int64  
 1   DEP         11857 non-null  int64  
 2   DEPCOM      11857 non-null  int64  
 3   DCIRIS      11857 non-null  object 
 4   AN          11857 non-null  int64  
 5   TYPEQU      11857 non-null  object 
 6   LAMBERT_X   11857 non-null  float64
 7   LAMBERT_Y   11857 non-null  float64
 8   QUALITE_XY  11857 non-null  object 
 9   LAB         11857 non-null  object 
 10  CLASSE      11857 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 1019.1+ KB


In [6]:
data.describe()

,REG,DEP,DEPCOM,AN,LAMBERT_X,LAMBERT_Y
count,11857.0,11857.0,11857.000000,11857.0,1.185700e+04,1.185700e+04
mean,93.0,6.0,6088.011976,2018.0,1.044110e+06,6.299014e+06
std,0.0,0.0,0.922078,0.0,9.442577e+02,9.973975e+02
min,93.0,6.0,6088.000000,2018.0,1.041556e+06,6.297580e+06
25%,93.0,6.0,6088.000000,2018.0,1.043406e+06,6.298264e+06
50%,93.0,6.0,6088.000000,2018.0,1.043953e+06,6.298733e+06
75%,93.0,6.0,6088.000000,2018.0,1.044912e+06,6.299542e+06
max,93.0,6.0,6159.000000,2018.0,1.048111e+06,6.303125e+06


In [7]:
features = list(data.columns)
features;

Transform iris code to integer value

In [8]:
data['iris']=data['DCIRIS'].apply(lambda x: int(x[:5]+x[6:])) 

In [9]:
data.drop(columns='DCIRIS',inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11857 entries, 0 to 11856
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   REG         11857 non-null  int64  
 1   DEP         11857 non-null  int64  
 2   DEPCOM      11857 non-null  int64  
 3   AN          11857 non-null  int64  
 4   TYPEQU      11857 non-null  object 
 5   LAMBERT_X   11857 non-null  float64
 6   LAMBERT_Y   11857 non-null  float64
 7   QUALITE_XY  11857 non-null  object 
 8   LAB         11857 non-null  object 
 9   CLASSE      11857 non-null  object 
 10  iris        11857 non-null  int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 1019.1+ KB


In [10]:
pd.unique(data['AN']) # observations only for 2008

array([2018])

In [11]:
pd.unique(data['QUALITE_XY']) # coordinates quality of X and Y

array(['Bonne', 'Mauvaise', 'Acceptable'], dtype=object)

In [12]:
data['QUALITE_XY'].value_counts()

Bonne         11598
Mauvaise        157
Acceptable      102
Name: QUALITE_XY, dtype: int64

### pb encoding dummy variables

In [13]:
enc = OneHotEncoder(sparse=False)

In [14]:
enc.fit(data[['QUALITE_XY']])

OneHotEncoder(sparse=False)

In [15]:
data['QUALITE_XY']

0           Bonne
1           Bonne
2           Bonne
3           Bonne
4        Mauvaise
           ...   
11852       Bonne
11853       Bonne
11854       Bonne
11855       Bonne
11856       Bonne
Name: QUALITE_XY, Length: 11857, dtype: object

In [16]:
quality_xy_encoded = enc.transform(data[['QUALITE_XY']])
quality_xy_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [17]:
data["Q_XY_acceptable"],data["Q_XY_bonne"],data['Q_XY_mauvaise']=quality_xy_encoded.T

In [18]:
data['Q_XY_bonne'].value_counts()

1.0    11598
0.0      259
Name: Q_XY_bonne, dtype: int64

In [19]:
data['Q_XY_mauvaise'].value_counts()

0.0    11700
1.0      157
Name: Q_XY_mauvaise, dtype: int64

In [20]:
data.head()

,REG,DEP,DEPCOM,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise
0,93,6,6088,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,60880801,0.0,1.0,0.0
1,93,6,6088,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,60881005,0.0,1.0,0.0
2,93,6,6088,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,60881005,0.0,1.0,0.0
3,93,6,6088,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,60883801,0.0,1.0,0.0
4,93,6,6088,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,60883801,0.0,0.0,1.0


### Numerical features

In [21]:
features_num = ['REG',
 'DEP',
 'DEPCOM',
 'iris',
 'AN','LAMBERT_X',
 'LAMBERT_Y']

In [22]:
data['TYPEQU']

0        F106
1        D233
2        G104
3        A505
4        A505
         ... 
11852    D237
11853    D242
11854    D301
11855    E101
11856    E101
Name: TYPEQU, Length: 11857, dtype: object

### Categorical features

In [23]:
features_cat = ['LAB','CLASSE','QUALITE_XY','TYPEQU']
features_cat

['LAB', 'CLASSE', 'QUALITE_XY', 'TYPEQU']

In [24]:
pd.unique(data['REG'])

array([93])

In [25]:
pd.unique(data['DEP'])

array([6])

In [26]:
pd.unique(data['DEPCOM'])

array([6088, 6159])

In [27]:
data['DEPCOM'].value_counts() # imbalance

6088    11855
6159        2
Name: DEPCOM, dtype: int64

In [28]:
pd.unique(data['iris']);

In [29]:
len(pd.unique(data['iris']))

110

In [30]:
len(pd.unique(data['LAMBERT_X']))

4915

In [31]:
data[features_cat]

,LAB,CLASSE,QUALITE_XY,TYPEQU
0,Centre équestre,Sport,Bonne,F106
1,Masseur kinésithérapeute D235,Santé,Bonne,D233
2,Information touristique,Tourisme,Bonne,G104
3,Agence immobilière,Services a,Bonne,A505
4,Agence immobilière,Services a,Mauvaise,A505
...,...,...,...,...
11852,Pédicure,Santé,Bonne,D237
11853,Diététicien,Santé,Bonne,D242
11854,Pharmacie,Santé,Bonne,D301
11855,Taxi,Transport,Bonne,E101


In [32]:
pd.unique(data['LAB'])

array(['Centre équestre', 'Masseur kinésithérapeute D235',
       'Information touristique', 'Agence immobilière',
       'Magasin de vêtements', 'Établissement santé court séjour',
       'Établissement santé moyen séjour',
       'Établissement santé long séjour', 'Psychologue', 'Pharmacie',
       'Hôtel', "Banque, Caisse d'épargne", 'Pompes funèbres',
       'Bureau de poste', 'Réparation auto et de matériel agicole',
       'Contrôle technique automobile', 'École de conduite',
       'Électricien', 'Entreprise générale du bâtiment', 'Coiffure',
       'Vétérinaire', 'Restaurant', 'Pressing', 'Institut de beauté',
       'Supermarché', 'Grande surface de bricolage', 'Supérette',
       'Épicerie', 'Boulangerie', 'Boucherie charcuterie',
       'Librairie papeterie journaux', "Magasin d'équipements du foyer",
       'Magasin de chaussures',
       "Magasin d'électroménager, audio vidéo B306",
       'Magasin de meubles',
       "Magasin d'art. de sports

In [33]:
data['LAB'].value_counts(ascending=True)

Maternité                                          1
Sports de glace                                     1
Tribunal de grande instance (TGI)                   1
Gare de voyageurs d’intérêt régional              1
Conseil départemental d'accès au droit (CDAD)       1
                                                 ... 
Coiffure                                          530
Magasin de vêtements                             620
Infirmier                                         692
Agence immobilière                               939
Restaurant                                       1917
Name: LAB, Length: 152, dtype: int64

In [34]:
len(pd.unique(data['TYPEQU'])) # unique values of equipement type

152

In [35]:
data['TYPEQU'].value_counts() # for meaning refer to 'LAB'

A504    1917
A505     939
D232     692
B302     620
A501     530
        ... 
F110       1
D606       1
F102       1
A107       1
A106       1
Name: TYPEQU, Length: 152, dtype: int64

In [36]:
len(pd.unique(data['LAB'])) # Equipement label

152

In [37]:
data['LAB'].value_counts()

Restaurant                                            1917
Agence immobilière                                    939
Infirmier                                              692
Magasin de vêtements                                  620
Coiffure                                               530
                                                      ... 
Centre lutte cancer                                      1
Gendarmerie                                              1
Port de plaisance                                        1
Centre équestre                                         1
Direction Régionale des Finances Publiques (DRFIP)       1
Name: LAB, Length: 152, dtype: int64

In [38]:
len(pd.unique(data['CLASSE']))

7

In [39]:
pd.unique(data['CLASSE']) # Equipement class
# Service a: a stands for 'alimentaire' ?

array(['Sport', 'Santé', 'Tourisme', 'Services a', 'Commerces',
       'Enseigneme', 'Transport'], dtype=object)

In [40]:
data['CLASSE'].value_counts() # pb of imbalance

Services a    5263
Santé         3125
Commerces     2403
Transport      443
Tourisme       310
Enseigneme     200
Sport          113
Name: CLASSE, dtype: int64

In [41]:
data.head()

,REG,DEP,DEPCOM,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise
0,93,6,6088,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,60880801,0.0,1.0,0.0
1,93,6,6088,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,60881005,0.0,1.0,0.0
2,93,6,6088,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,60881005,0.0,1.0,0.0
3,93,6,6088,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,60883801,0.0,1.0,0.0
4,93,6,6088,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,60883801,0.0,0.0,1.0


In [42]:
data

,REG,DEP,DEPCOM,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise
0,93,6,6088,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,60880801,0.0,1.0,0.0
1,93,6,6088,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,60881005,0.0,1.0,0.0
2,93,6,6088,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,60881005,0.0,1.0,0.0
3,93,6,6088,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,60883801,0.0,1.0,0.0
4,93,6,6088,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,60883801,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11852,93,6,6088,2018,D237,1.045557e+06,6298700.200,Bonne,Pédicure,Santé,60880304,0.0,1.0,0.0
11853,93,6,6088,2018,D242,1.045562e+06,6298868.500,Bonne,Diététicien,Santé,60880304,0.0,1.0,0.0
11854,93,6,6088,2018,D301,1.045551e+06,6298723.400,Bonne,Pharmacie,Santé,60880304,0.0,1.0,0.0
11855,93,6,6088,2018,E101,1.045523e+06,6298718.500,Bonne,Taxi,Transport,60880304,0.0,1.0,0.0


### Attempt to group the quartiers by status (luxe, city center, commercial district, defavorable...)

In [56]:
data.groupby('iris').nunique()

,REG,DEP,DEPCOM,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise
iris,,,,,,,,,,,,,
60880101,1,1,1,1,52,143,139,3,52,6,2,2,2
60880102,1,1,1,1,37,160,156,3,37,6,2,2,2
60880103,1,1,1,1,45,111,111,2,45,7,2,2,1
60880201,1,1,1,1,59,96,94,3,59,7,2,2,2
60880202,1,1,1,1,43,60,59,3,43,7,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60883301,1,1,1,1,1,1,1,1,1,1,1,1,1
60883501,1,1,1,1,1,1,1,1,1,1,1,1,1
60883701,1,1,1,1,4,4,4,2,4,2,2,2,1


Centre ville :
 'Information touristique'
 'Agence de voyages'
 'Taxi'
 'Gare de voyageurs d’intérêt national E108'
 'Gare de voyageurs d’intérêt régional'
 'Cinema' (centres comm?)
 
Luxe :
 'Centre équestre'
 'Théâtre'
 'Port de plaisance'
 'Horlogerie Bijouterie', 'Fleuriste', 'Magasin d’optique
 'Restaurant'
 'Parfumerie'
 'Coiffure'
 'Hôtel'
 'Agence immobilière'
 
Industriel :
 'Contrôle technique automobile'
 'Station service'
 
 
 'Police','Gendarmerie'

In [ ]:
# Mapping of each lab type into a caterogy within a dict
# apply mapping on the lab column in creating a new one

# dict_luxe={'horlogerie':'luxe',..}
# data['category']=dat['lab'].map(dict_categ)

In [57]:
industrial =[ 'Contrôle technique automobile', 'Station service']

In [60]:
luxe = ['Centre équestre',
         'Théâtre',
         'Port de plaisance',
         'Horlogerie Bijouterie']

commercial = ['Magasin d’optique',
              'Coiffure',
              'Fleuriste', 
              'Restaurant',
              'Parfumerie',
              'Hôtel',
              'Agence immobilière']

tourist = ['Information touristique',
            'Agence de voyages',
            'Taxi',
            'Gare de voyageurs d’intérêt national E108',
            'Gare de voyageurs d’intérêt régional']
police = ['Police','Gendarmerie']

In [66]:
 def count_industrial_labs(s):
        return s.map(lambda x: 1 if x in  industriel else 0).sum()

def count_luxe_labs(s):
    return s.map(lambda x: 1 if x in  luxe else 0).sum()

 def count_commercial_labs(s):
        return s.map(lambda x: 1 if x in  commercial else 0).sum()

def count_tourist_labs(s):
    return s.map(lambda x: 1 if x in  tourist else 0).sum()
def count_police_labs(s):
    return s.map(lambda x: 1 if x in police else 0).sum()

In [75]:
df=data.groupby('iris').agg({
    'LAB': [count_industrial_labs, count_luxe_labs, count_commercial_labs,count_tourist_labs],
    })
df

LAB                                        \
         count_industrial_labs count_luxe_labs count_commercial_labs   
iris                                                                   
60880101                     0               3                   163   
60880102                     0               6                   163   
60880103                     0               2                    82   
60880201                     1               2                    74   
60880202                     2               2                    68   
...                        ...             ...                   ...   
60883301                     0               0                     0   
60883501                     0               0                     0   
60883701                     0               0                     0   
60883801                     0               1                    26   
61590101                     1               0                     0   

                             
         count_tourist_labs  
iris                         
60880101                  3  
60880102                  2  
60880103                  3  
60880201                  9  
60880202                 10  
...                     ...  
60883301                  0  
60883501                  0  
60883701                  0  
60883801                  7  
61590101                  0  

[110 rows x 4 columns]

In [78]:
data_g=data.merge(df,how='inner',on='iris')

/home/mchistya/.pyenv/versions/3.8.6/envs/trampact/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [79]:
data_g.groupby('iris').count()

,REG,DEP,DEPCOM,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise,"(LAB, count_industrial_labs)","(LAB, count_luxe_labs)","(LAB, count_commercial_labs)","(LAB, count_tourist_labs)"
iris,,,,,,,,,,,,,,,,,
60880101,275,275,275,275,275,275,275,275,275,275,275,275,275,275,275,275,275
60880102,289,289,289,289,289,289,289,289,289,289,289,289,289,289,289,289,289
60880103,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187,187
60880201,273,273,273,273,273,273,273,273,273,273,273,273,273,273,273,273,273
60880202,155,155,155,155,155,155,155,155,155,155,155,155,155,155,155,155,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60883301,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
60883501,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
60883701,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
